In [14]:
import pandas as pd
import numpy as np
from plotly.graph_objs import *
import plotly.plotly as py
import plotly.graph_objs as go
import cmocean
import matplotlib.pyplot as plt

In [2]:
ccn_data = pd.read_csv('CCN_0.5-NaN-time.csv', sep=',',skiprows=0,header=0,na_values=[9999])

In [3]:
ccn_data.head()

,AMR_lowest,AMR_surf,AOD,B4LI,BJC,CAPE,CAPE_lowest,CAPE_nc,CAPE_surf,CCN_0.5_cm3,...,row,satz,solz,sound_D(km),sound_ID,sound_hour,sumN_0.5,sumN_0.5_sumN_1.0,sumN_1.0,winNO
0,NaN,NaN,NaN,-3.6,2013-06-01 05:09:00,1119.0,NaN,667,NaN,94.0,...,6112,48.06,23.20,NaN,NaN,NaN,891.0,1.01,880.0,501
1,NaN,NaN,NaN,-2.8,2013-06-01 05:09:00,559.0,NaN,941,NaN,468.0,...,5920,31.90,20.35,NaN,NaN,NaN,306.0,1.00,306.0,708
2,NaN,NaN,NaN,-3.2,2013-06-01 05:09:00,812.0,NaN,941,NaN,688.0,...,5920,28.89,19.91,NaN,NaN,NaN,1652.0,1.00,1652.0,709
3,NaN,NaN,NaN,-6.9,2013-06-01 05:09:00,2822.0,NaN,1904,NaN,1337.0,...,6112,-0.48,16.26,NaN,NaN,NaN,171.0,1.00,171.0,1101
4,NaN,NaN,NaN,-6.9,2013-06-01 05:09:00,2822.0,NaN,1904,NaN,1538.0,...,6112,-3.38,15.92,NaN,NaN,NaN,514.0,1.00,514.0,1102


In [4]:
data=ccn_data.set_index('BJC')

In [5]:
data1=data

In [6]:
step = 0.5
to_bin = lambda x: np.floor(x / step) * step
data1["lat_bin"] = data1.lat.map(to_bin)
data1["lon_bin"] = data1.lon.map(to_bin)
groups = data1.groupby(("lat_bin", "lon_bin"))
#avg_data=all_data.set_index(['latbin', 'lonbin']).stack().groupby(level=[0, 1]).mean()
avg_data=data1.groupby(['lat_bin','lon_bin']).count()
print(avg_data.head())

                 AMR_lowest  AMR_surf  AOD  B4LI  CAPE  CAPE_lowest  CAPE_nc  \
lat_bin lon_bin                                                                
7.0     124.5             0         0    0     2     2            0        2   
        125.5             0         0    0     4     4            0        4   
7.5     124.0             0         0    0     8     8            0        8   
        125.0             0         0    0     2     2            0        2   
8.0     123.5             0         0    0     2     2            0        2   

                 CAPE_surf  CCN_0.5_cm3  CCN_0.5_mg  ...  row  satz  solz  \
lat_bin lon_bin                                      ...                    
7.0     124.5            0            2           2  ...    2     2     2   
        125.5            0            4           4  ...    4     4     4   
7.5     124.0            0            8           8  ...    8     8     8   
        125.0            0            2           2  .

C:\Users\xigrug\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """


In [7]:
avg_data.to_csv("count_CCN1.csv",index=True,sep=',')

In [9]:
mapbox_access_token = 'pk.eyJ1IjoieGlncnVnIiwiYSI6ImNqZm8ycXMxejAwOXYyenM3Z2twOWJvb2EifQ.lSikTXoOmtTjHutewtCe9A'
#mapbox_access_token = 'pk.eyJ1Ijoiam1tZWFzZSIsImEiOiJjamljeWkwN3IwNjEyM3FtYTNweXV4YmV0In0.2zbgGCjbPTK7CToIg81kMw'

In [10]:
def cmocean_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        # https://stackoverflow.com/questions/6800481/python-map-object-is-not-subscriptable
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

In [11]:
def plotMap(lat,lon,ccn,ss):
    data = Data([
    go.Scattermapbox(
        # 绘制散点的经纬度
            lat=lat,
            lon=lon,
            mode='markers',
            marker=Marker(
                size=8,
                color=ccn,
                showscale=True,
                cmax=150,
                cmin=10,
                colorscale=curl
            ),
            # 散点对应的文本
            text=ccn,
            customdata=ss,
            name='CCN'
        )
    ])
    layout = Layout(
        title='基于mapbox的地图标注',
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            domain={'x': [0, 1.0], 'y': [0, 1]},
            bearing=0,
            # 地图中心坐标，不要远离绘制的散点坐标
            center=dict(
                lat=32,
                lon=118
            ),
            pitch=0,
            zoom=4
        ),
    )
    fig = go.FigureWidget(data=data, layout=layout)
    return fig

In [12]:
curl = cmocean_to_plotly(cmocean.cm.curl, 20)

In [15]:
# Extract the data we're interested in
lat=avg_data.index.get_level_values(0)
lon=avg_data.index.get_level_values(1)
ccn = avg_data['CCN_1.0_cm3'].values
ss = avg_data['SS_1.0_mg'].values
lat_max=np.nanmax(lat)
lon_max=np.nanmax(lon)
lat_min=np.nanmin(lat)
lon_min=np.nanmin(lon)
ccn_max=np.nanmax(ccn)
print(ccn_max)
ccn_min=np.nanmin(ccn)
print(ccn_min)

fig= plotMap(lat,lon,ccn,ss)
plt.savefig('count_CCN.png',dpi=200)

141
1


<Figure size 432x288 with 0 Axes>

In [16]:
fig

FigureWidget({
    'data': [{'customdata': array([2, 4, 8, ..., 1, 1, 1], dtype=int64),
              'lat': a…

In [17]:
data_10=avg_data[avg_data['CCN_1.0_cm3']>=10]

In [18]:
ccn= data_10['CCN_1.0_cm3'].values
ss= data_10['SS_1.0_mg'].values

In [19]:
lat=data_10.index.get_level_values(0)
lon=data_10.index.get_level_values(1)

In [20]:
lat_max=np.nanmax(lat)
lon_max=np.nanmax(lon)
lat_min=np.nanmin(lat)
lon_min=np.nanmin(lon)
ccn_max=np.nanmax(ccn)
print(ccn_max)
ccn_min=np.nanmin(ccn)
print(ccn_min)

141
10


In [21]:
fig= plotMap(lat,lon,ccn,ss)
plt.savefig('count_CCN10.png',dpi=200)

<Figure size 432x288 with 0 Axes>

In [22]:
fig

FigureWidget({
    'data': [{'customdata': array([10, 10, 18, ..., 11, 10, 10], dtype=int64),
              'l…